# Tensors
Some experiments on stuff that I found interesting or that was new to me.

## Preamble

In [2]:
import torch

## Playground

Skalar entries of `Tensor`s are `Tensor`s, as well.

In [3]:
a = torch.ones(3)
print("first element: ", a[0])
print("as float:      ", float(a[0]))

first element:  tensor(1.)
as float:       1.0


Adding dimensions (aka dirty `unsqueeze`):

In [4]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
print(points.shape)
print(points[None].shape)

torch.Size([3, 2])
torch.Size([1, 3, 2])


### Einstein Summation (`torch.einsum`)
A dense, powerful DSL to combine tensors. See [block post by Tim Rockäschel](https://rockt.github.io/2018/04/30/einsum) for more.

Skalar product: $c = \sum_i a_i * b_i$

In [5]:
a = torch.ones(3)
b = torch.ones(3)
torch.einsum('i,i->', [a, b])

tensor(3.)

Tensor contraction with a transpose: $c_i = \sum_{k} \sum_{l} A_{ikl} B_{lk}$

In [6]:
A = torch.randn(2, 3, 4)
B = torch.randn(4, 3)
torch.einsum('ikl,lk->i', [A, B])


tensor([-1.5601, -3.2550])

### Named Tensors
We can give names to the dimensions of a tensor. This is experimental but very promising, the names are considered when broadcasting and mismatched dimensions throw an error.

Naming tensors:

In [81]:
named_tensor = torch.tensor([[1, 2], [3, 4]], names=['rows', 'columns'])
print(f"named_tensor:\n{named_tensor}\n")

unnamed_tensor = torch.zeros([1,2,3])
renamed_tensor = unnamed_tensor.refine_names(..., 'C', 'H', 'W')
print(f"renamed_tensor:\n{renamed_tensor}")

named_tensor:
tensor([[1, 2],
        [3, 4]], names=('rows', 'columns'))

renamed_tensor:
tensor([[[0., 0., 0.],
         [0., 0., 0.]]], names=('C', 'H', 'W'))


Mismatched dimensions throw an error:

In [82]:
A = torch.ones(2,2, names=['H', 'W'])
B = torch.ones(2,2, names=['W', 'H'])

#A * B  # error

Explicitly re-aligning resolves the issue:

In [30]:
A * B.align_as(A)

tensor([[1., 1.],
        [1., 1.]], names=('H', 'W'))

Currently Einstein summation is not supported with named tensors (drop names via `A.rename(None)`).

In [83]:
#torch.einsum('ij,ij->ij', [A, B])  # error
torch.einsum('ij,ij->ij', [A.rename(None), B.rename(None)])  # fine

tensor([[1., 1.],
        [1., 1.]])

### Storage
Tensors are wrappers around a storage in which the entries are stored flat. Higher dimensionalities are achieved by `shape` and `stride` attribute. This enables some-shaping operations to be efficient by simply producing new views into the same storage.

In [71]:
tensor = torch.tensor(((1,1,1), (2,2,2)))
print(f"tensor:\n{tensor}\n")
print(f"storage:\n{tensor.storage()}\n")
print(f"shape:\n{tensor.shape}\n")
print(f"stride:\n{tensor.stride()}\n")
print(f"transposed stride:\n{tensor.t().stride()}\n")
print("tensor's storage == transposed tensor's storage:\n"
      f"{tensor.storage().data_ptr() == tensor.t().storage().data_ptr()}\n")

tensor:
tensor([[1, 1, 1],
        [2, 2, 2]])

storage:
 1
 1
 1
 2
 2
 2
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]

shape:
torch.Size([2, 3])

stride:
(3, 1)

transposed stride:
(1, 3)

tensor's storage == transposed tensor's storage:
True



This might lead to `Tensor`s that are non-contiguous. Some pytorch operations require contiguous memory layout which can be achieved with `.contiguous()` which returns a new `Tensor` with its own, contiguous storage.

## Tensor computation too slow?

Have a look at [tensor comprehensions](https://pytorch.org/blog/tensor-comprehensions/) to conveniently write custom GPU code.